# **Dataset cleaning**

In [69]:
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
from fc_cleaning import *

In [70]:
client = MongoClient("mongodb://localhost/datamad0320")
db = client.get_database()

In [71]:
# Getting dataset from MongoDB
all_companies = db.companies.find({},{"name":1,"offices":1,"total_money_raised":1,"number_of_employees":1,
                                   "founded_year":1,"category_code":1})

In [72]:
df = pd.DataFrame(all_companies)
display(df.head(),df.shape)

,_id,name,category_code,number_of_employees,founded_year,total_money_raised,offices
0,52cdef7c4bab8bd675297d8b,AdventNet,enterprise,600.0,1996.0,$0,"[{'description': 'Headquarters', 'address1': '..."
1,52cdef7c4bab8bd675297d92,Flektor,games_video,NaN,NaN,$0,"[{'description': None, 'address1': '8536 Natio..."
2,52cdef7c4bab8bd675297d8c,Zoho,software,1600.0,2005.0,$0,"[{'description': 'Headquarters', 'address1': '..."
3,52cdef7c4bab8bd675297d8a,Wetpaint,web,47.0,2005.0,$39.8M,"[{'description': '', 'address1': '710 - 2nd Av..."
4,52cdef7c4bab8bd675297d93,Fox Interactive Media,web,0.0,1979.0,$0,"[{'description': '', 'address1': '407 N Maple ..."


(18801, 7)

In [73]:
# Changing "_id" to json format to not get errors when they are saved into a json file
for i in range(len(df["_id"])):
    df["_id"].iloc[i] = dumps(df["_id"].iloc[i])

/home/flori/.local/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


### **Split column "offices" by the offices which contains**

In [74]:
# using explode() function to split the column "offices" in the different offices that are containing in this array
df = df.explode("offices")
display(df.head(),df.shape)

,_id,name,category_code,number_of_employees,founded_year,total_money_raised,offices
0,"{""$oid"": ""52cdef7c4bab8bd675297d8b""}",AdventNet,enterprise,600.0,1996.0,$0,"{'description': 'Headquarters', 'address1': '4..."
1,"{""$oid"": ""52cdef7c4bab8bd675297d92""}",Flektor,games_video,NaN,NaN,$0,"{'description': None, 'address1': '8536 Nation..."
2,"{""$oid"": ""52cdef7c4bab8bd675297d8c""}",Zoho,software,1600.0,2005.0,$0,"{'description': 'Headquarters', 'address1': '4..."
3,"{""$oid"": ""52cdef7c4bab8bd675297d8a""}",Wetpaint,web,47.0,2005.0,$39.8M,"{'description': '', 'address1': '710 - 2nd Ave..."
3,"{""$oid"": ""52cdef7c4bab8bd675297d8a""}",Wetpaint,web,47.0,2005.0,$39.8M,"{'description': '', 'address1': '270 Lafayette..."


(21762, 7)

#### 1. Name of cities where offices are located

In [75]:
#function to get city names where offices are located
city = pd.DataFrame(df.apply(getCity, axis=1))
city.columns=["city"]
city.head()

,city
0,Pleasanton
1,Culver City
2,Pleasanton
3,Seattle
3,New York


In [76]:
df = pd.concat([df,city], axis=1)
display(df.head(),df.shape)

,_id,name,category_code,number_of_employees,founded_year,total_money_raised,offices,city
0,"{""$oid"": ""52cdef7c4bab8bd675297d8b""}",AdventNet,enterprise,600.0,1996.0,$0,"{'description': 'Headquarters', 'address1': '4...",Pleasanton
1,"{""$oid"": ""52cdef7c4bab8bd675297d92""}",Flektor,games_video,NaN,NaN,$0,"{'description': None, 'address1': '8536 Nation...",Culver City
2,"{""$oid"": ""52cdef7c4bab8bd675297d8c""}",Zoho,software,1600.0,2005.0,$0,"{'description': 'Headquarters', 'address1': '4...",Pleasanton
3,"{""$oid"": ""52cdef7c4bab8bd675297d8a""}",Wetpaint,web,47.0,2005.0,$39.8M,"{'description': '', 'address1': '710 - 2nd Ave...",Seattle
3,"{""$oid"": ""52cdef7c4bab8bd675297d8a""}",Wetpaint,web,47.0,2005.0,$39.8M,"{'description': '', 'address1': '270 Lafayette...",New York


(21762, 8)

#### 2. Location of offices (GeoPoint)

In [77]:
# transform office object into GeoPoint for office
cleaned_offices = df.apply(officeToGeoPoint, axis=1, result_type="expand")
cleaned_offices.columns = ["location","state"]
display(cleaned_offices.head(),cleaned_offices.shape)

,location,state
0,"{'type': 'Point', 'coordinates': [-121.904945,...",success
1,"{'type': 'Point', 'coordinates': [-118.379768,...",success
2,"{'type': 'Point', 'coordinates': [-121.904945,...",success
3,"{'type': 'Point', 'coordinates': [-122.333253,...",success
3,"{'type': 'Point', 'coordinates': [-73.9964312,...",success


(21762, 2)

In [78]:
df = pd.concat([df,cleaned_offices], axis=1)
display(df.head(),df.shape)

,_id,name,category_code,number_of_employees,founded_year,total_money_raised,offices,city,location,state
0,"{""$oid"": ""52cdef7c4bab8bd675297d8b""}",AdventNet,enterprise,600.0,1996.0,$0,"{'description': 'Headquarters', 'address1': '4...",Pleasanton,"{'type': 'Point', 'coordinates': [-121.904945,...",success
1,"{""$oid"": ""52cdef7c4bab8bd675297d92""}",Flektor,games_video,NaN,NaN,$0,"{'description': None, 'address1': '8536 Nation...",Culver City,"{'type': 'Point', 'coordinates': [-118.379768,...",success
2,"{""$oid"": ""52cdef7c4bab8bd675297d8c""}",Zoho,software,1600.0,2005.0,$0,"{'description': 'Headquarters', 'address1': '4...",Pleasanton,"{'type': 'Point', 'coordinates': [-121.904945,...",success
3,"{""$oid"": ""52cdef7c4bab8bd675297d8a""}",Wetpaint,web,47.0,2005.0,$39.8M,"{'description': '', 'address1': '710 - 2nd Ave...",Seattle,"{'type': 'Point', 'coordinates': [-122.333253,...",success
3,"{""$oid"": ""52cdef7c4bab8bd675297d8a""}",Wetpaint,web,47.0,2005.0,$39.8M,"{'description': '', 'address1': '270 Lafayette...",New York,"{'type': 'Point', 'coordinates': [-73.9964312,...",success


(21762, 10)

#### 3. Latitude and longitude from the GeoPoint values

In [79]:
## function to get latitude and longitude values from GeoPoint location
lat_long = df.apply(easyLatLng,axis=1,result_type="expand")
lat_long.head()
df = pd.concat([df,lat_long],axis=1)

### **Cleaning of column "total money raised"**

In [80]:
# function to group total_money_raised values by currency and units (only to see how many different values we have)
#print(list(company_processed["total_money_raised"].value_counts().index))
df["total_money_raised_cleaning"] = df["total_money_raised"].apply(getTypeMoney)

In [81]:
df["total_money_raised_cleaning"].value_counts()

$      15527
$M      5059
$k       705
€M       201
€k       104
£M        80
£k        37
C$M       19
$B        15
C$k        9
¥M         3
¥B         2
krM        1
Name: total_money_raised_cleaning, dtype: int64

In [82]:
# function to transform total_money_raised values into $ an k units    
money_conversion = df.apply(getMoney,axis=1,result_type="expand")
money_conversion.columns=["total_money_raised ($k)","money_state"]
money_conversion.head()

,total_money_raised ($k),money_state
0,0.0,success
1,0.0,success
2,0.0,success
3,39800.0,success
3,39800.0,success


In [83]:
display(money_conversion.shape)
money_conversion.money_state.value_counts()

(21762, 2)

success    21762
Name: money_state, dtype: int64

In [84]:
df = pd.concat([df,money_conversion], axis=1)
display(df.head(),df.shape)

,_id,name,category_code,number_of_employees,founded_year,total_money_raised,offices,city,location,state,latitude,longitude,total_money_raised_cleaning,total_money_raised ($k),money_state
0,"{""$oid"": ""52cdef7c4bab8bd675297d8b""}",AdventNet,enterprise,600.0,1996.0,$0,"{'description': 'Headquarters', 'address1': '4...",Pleasanton,"{'type': 'Point', 'coordinates': [-121.904945,...",success,37.6929,-121.905,$,0.0,success
1,"{""$oid"": ""52cdef7c4bab8bd675297d92""}",Flektor,games_video,NaN,NaN,$0,"{'description': None, 'address1': '8536 Nation...",Culver City,"{'type': 'Point', 'coordinates': [-118.379768,...",success,34.026,-118.38,$,0.0,success
2,"{""$oid"": ""52cdef7c4bab8bd675297d8c""}",Zoho,software,1600.0,2005.0,$0,"{'description': 'Headquarters', 'address1': '4...",Pleasanton,"{'type': 'Point', 'coordinates': [-121.904945,...",success,37.6929,-121.905,$,0.0,success
3,"{""$oid"": ""52cdef7c4bab8bd675297d8a""}",Wetpaint,web,47.0,2005.0,$39.8M,"{'description': '', 'address1': '710 - 2nd Ave...",Seattle,"{'type': 'Point', 'coordinates': [-122.333253,...",success,47.6031,-122.333,$M,39800.0,success
3,"{""$oid"": ""52cdef7c4bab8bd675297d8a""}",Wetpaint,web,47.0,2005.0,$39.8M,"{'description': '', 'address1': '270 Lafayette...",New York,"{'type': 'Point', 'coordinates': [-73.9964312,...",success,40.7237,-73.9964,$M,39800.0,success


(21762, 15)

### **Removing unnecessary columns**

In [85]:
df_clean = df.drop(columns=["total_money_raised_cleaning","offices","total_money_raised","money_state","state"])

In [86]:
display(df_clean.head(),df_clean.shape)

,_id,name,category_code,number_of_employees,founded_year,city,location,latitude,longitude,total_money_raised ($k)
0,"{""$oid"": ""52cdef7c4bab8bd675297d8b""}",AdventNet,enterprise,600.0,1996.0,Pleasanton,"{'type': 'Point', 'coordinates': [-121.904945,...",37.6929,-121.905,0.0
1,"{""$oid"": ""52cdef7c4bab8bd675297d92""}",Flektor,games_video,NaN,NaN,Culver City,"{'type': 'Point', 'coordinates': [-118.379768,...",34.026,-118.38,0.0
2,"{""$oid"": ""52cdef7c4bab8bd675297d8c""}",Zoho,software,1600.0,2005.0,Pleasanton,"{'type': 'Point', 'coordinates': [-121.904945,...",37.6929,-121.905,0.0
3,"{""$oid"": ""52cdef7c4bab8bd675297d8a""}",Wetpaint,web,47.0,2005.0,Seattle,"{'type': 'Point', 'coordinates': [-122.333253,...",47.6031,-122.333,39800.0
3,"{""$oid"": ""52cdef7c4bab8bd675297d8a""}",Wetpaint,web,47.0,2005.0,New York,"{'type': 'Point', 'coordinates': [-73.9964312,...",40.7237,-73.9964,39800.0


(21762, 10)

# **Analysis to choose a city**

#### **First, cities where there are companies doing design**

In [87]:
df_clean["city"][df_clean["category_code"]=="design"].value_counts()

San Francisco    1
Berlin           1
Collingwood      1
Ellensburg       1
London           1
Brooklyn         1
Name: city, dtype: int64

#### **Next, cities where more tech startups exist with a "total_money_raised" greater than or equal to $1M. The "founded_year" of the company has to be after 2009**

In [88]:
# "founded_year" after 2009 because this dataset does not contain a company whose "founded_year" is after 2013 (this dataset may 
# be from 2014). So, "founded_year" will be 3 years before 2013
df_clean["city"][((df_clean["category_code"]=="web")|(df_clean["category_code"]== "software")|
                  (df_clean["category_code"]=="games_video")|(df_clean["category_code"]=="mobile")|
                  (df_clean["category_code"]=="network_hosting")|(df_clean["category_code"]=="cleantech")|
                  (df_clean["category_code"]=="biotech")|(df_clean["category_code"]=="photo_video")|
                  (df_clean["category_code"]=="hardware")|(df_clean["category_code"]=="messaging"))
                 & (df_clean["founded_year"]>=2010) & (df_clean["total_money_raised ($k)"]>=float(1000))].value_counts()

San Francisco          3
Hopkinton              2
Scottsdale             2
NoOffice               2
Denver                 2
Los Altos              1
SOUTH BOSTON           1
Santa Monica           1
Bangalore              1
Tel Aviv               1
Waterloo, ON           1
San Jose               1
South San Francisco    1
London                 1
New York               1
San Mateo              1
Fremont                1
Los Angeles            1
Palo Alto              1
Beverly Hills          1
NoCity                 1
Santa Clara            1
Name: city, dtype: int64

#### **Finally, cities classified by number companies (or offices)**

In [89]:
df_clean["city"].value_counts()

NoOffice         5057
San Francisco     906
New York          837
NoCity            746
London            616
                 ... 
Lysaker             1
Wrexham             1
Montrose            1
Chatham             1
Goa                 1
Name: city, Length: 3126, dtype: int64

#### **San Francisco is the city with more companies, with a design company and with more successful tech startups. So, San Francisco is going to be the city where the company will be located**

In [90]:
# Choosing companies (or offices) located in San Francisco
companiesSF = df_clean[df_clean["city"]=="San Francisco"]

In [92]:
# Removing "_id" column because several "_id" are repeated (due to the explode() function) and MongoDB does not import documents 
# with the same "_id"
# Also, removing unncessary coolumns
companiesSF= companiesSF.drop(columns=["_id","city"])

In [94]:
companiesSF= companiesSF.reset_index()

In [96]:
companiesSF= companiesSF.drop(columns="index")

In [97]:
display(companiesSF.head(),companiesSF.shape)

,name,category_code,number_of_employees,founded_year,location,latitude,longitude,total_money_raised ($k)
0,Digg,news,60.0,2004.0,"{'type': 'Point', 'coordinates': [-122.394523,...",37.7647,-122.395,45000.0
1,Scribd,news,50.0,2007.0,"{'type': 'Point', 'coordinates': [-122.404052,...",37.7896,-122.404,25800.0
2,StumbleUpon,web,NaN,2002.0,"{'type': 'Point', 'coordinates': [-122.419204,...",37.7752,-122.419,18500.0
3,Twitter,social,1300.0,2006.0,"{'type': 'Point', 'coordinates': [-122.4169244...",37.7768,-122.417,1160000.0
4,Powerset,search,60.0,2006.0,"{'type': 'Point', 'coordinates': [-122.395289,...",37.7786,-122.395,22500.0


(906, 8)

### **Save clean dataset**

In [98]:
companiesSF.to_json("../output/companiesSF.json",orient="records")
companiesSF.to_csv("../output/companiesSF.csv")